In [4]:
import plotly.express as px
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path

import pandas as pd
from pathlib import Path

data_dir=Path('/Users/suzinyou/Dropbox/가천대 서울연구원 프로젝트') / '10. 데이터산출물' / '09_older_adults'

In [51]:
d = pd.read_excel(data_dir / 'older_adults.xlsx', sheet_name=None)

In [52]:
keys = list(d.keys())
keys

['1) older_adults_demogr(seoul)',
 '2) indi_income_by_src',
 '3) income_by_pnsn_status',
 '4) eq_inc_by_hh_type']

In [16]:
from src.data.utils import load_cpi, load_sido_names, load_sigungu_names
from src.dictionary import translate
cpi_seoul = load_cpi(translate('seoul'))
cpi_kr = load_cpi(translate('kr'))

In [28]:
cpi_seoul['region'] = 'seoul'
cpi_kr['region'] = 'kr'

In [29]:
cpi = pd.concat([cpi_seoul, cpi_kr])

In [53]:
for k in keys:
    d[k] = d[k].rename(columns={col:col.lower() for col in d[keys[0]].columns})

In [54]:
for i in range(1, 4):
    df = d[keys[i]]
    df = df.merge(cpi, on=['std_yyyy', 'region'])
    for col in df.columns:
        if 'mean' in col or 'median' in col:
            df[col + "_real"] = df[col] / df.cpi
    d[keys[i]] = df

In [55]:
with pd.ExcelWriter(data_dir / 'older_adults_processed.xlsx') as writer:
    for k, df in d.items():
        df.to_excel(writer, index=False, sheet_name=k)